In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

In [3]:
dataset_path = "C:\main project\kvasir-dataset-v2"

In [4]:
img_height, img_width = 224, 224
batch_size = 32
validation_split = 0.2

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=validation_split)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')


Found 6400 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.


In [7]:
base_model = NASNetLarge(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))




343610240/343610240 [==============================] - 23s 0us/step


In [8]:
# custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

In [9]:
# Combine base model and custom head
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [11]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

Epoch 1/20



200/200 [==============================] - 772s 4s/step - loss: 0.7244 - accuracy: 0.7198 - val_loss: 0.5640 - val_accuracy: 0.7700
Epoch 2/20
200/200 [==============================] - 584s 3s/step - loss: 0.5033 - accuracy: 0.7977 - val_loss: 0.4937 - val_accuracy: 0.8031
Epoch 3/20
200/200 [==============================] - 356s 2s/step - loss: 0.4604 - accuracy: 0.8123 - val_loss: 0.5645 - val_accuracy: 0.7788
Epoch 4/20
200/200 [==============================] - 364s 2s/step - loss: 0.4179 - accuracy: 0.8284 - val_loss: 0.5031 - val_accuracy: 0.8081
Epoch 5/20
200/200 [==============================] - 364s 2s/step - loss: 0.3912 - accuracy: 0.8380 - val_loss: 0.4909 - val_accuracy: 0.8019
Epoch 6/20
200/200 [==============================] - 367s 2s/step - loss: 0.3801 - accuracy: 0.8461 - val_loss: 0.4976 - val_accuracy: 0.7869
Epoch 7/20
200/200 [==============================] - 370s 2s/step - loss: 0.3652 - accuracy: 0.8430 - val_loss: 0.5175 - val_accuracy: 0.8031
Epoch 8/20

In [13]:
# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('\nTest accuracy:', test_acc)

50/50 - 71s - loss: 0.5106 - accuracy: 0.7994 - 71s/epoch - 1s/step

Test accuracy: 0.7993749976158142
